In [21]:
## OpenSearch 벡터 스토어 및 Amazon Bedrock을 사용한 RAG

In [22]:
!pip list | grep langchain
!pip list | grep opensearch

langchain                     0.0.345
langchain-core                0.0.9
langchain-experimental        0.0.42
opensearch-dsl                2.1.0
opensearch-py                 2.3.2


In [23]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [24]:
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=None,
  endpoint_url=None,
  region="us-east-1"
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## 2. Titan Embedding 및 LLM Claude-v2 모델 로딩
### claude-v2 model loading

In [25]:

from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f5104be2b30>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f5104bdf3a0>])

### Embedding 모델 로딩

In [26]:
from langchain.embeddings.bedrock import BedrockEmbeddings
llm_emb = BedrockEmbeddings(
  model_id="amazon.titan-embed-text-v1",
  client=boto3_bedrock,
  region_name='us-east-1'
)

### 데이터 준비

In [27]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df.head()

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### 데이터 전처리
- column no 제거
- 기존 Category column name -> ask

In [28]:
pre_df = df.drop(columns=['no'], axis=1)
pre_df.rename(columns={"Category":"ask"}, inplace=True)
print_ww(pre_df.head())
pre_df.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=False)

                            ask  \
0   아마존 은행의 타기관OTP 이용등록방법 알려주세요
1  아마존 공동인증서와 금융인증서 차이점이 무엇인가요?
2      공동인증서와 금융인증서 차이점이 무엇인가요?
3           타기관OTP 이용등록방법 알려주세요
4      공동인증서와 금융인증서 차이점이 무엇인가요?

                                         Information   type Source
0  아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...  인터넷뱅킹  아마존은행
1  공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...    인증서  아마존은행
2  공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...    인증서   신한은행
3  타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....  인터넷뱅킹   신한은행
4  공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...    인증서   신한은행


### CSV 문서 로딩

In [29]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(
  file_path="data/fsi_smart_faq_ko_preprocess.csv",
  source_column="Source",
  encoding="utf-8",
)

### 메타 데이터 생성
- 컬럼의 type, source는 metadata로 생성하고, 내용에서는 삭제
- 타임스탬프, 임베딩 모델의 endpoint 이름을 metadata로 추가

In [30]:
import time
documents_fsi = loader.load()

In [31]:
documents_fsi[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 아마존은행', metadata={'source': '아마존은행', 'row': 0})

In [32]:
def create_metadata(docs):
  for idx, doc in enumerate(docs):
    # type 을 metadata로 저장
    split_content = doc.page_content.split("type: ")
    content = split_content[0]
    type = split_content[1].split("\n")[0]
    doc.page_content = content
    doc.metadata['type'] = type
    doc.metadata['timestamp'] = time.time()

create_metadata(documents_fsi)

In [33]:
documents_fsi[0]

Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1701838934.8790963})

### Text Splitter 로 chunking

In [34]:
chunk_size = 2048
chunk_overlap = 0

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=chunk_size,
  chunk_overlap=chunk_overlap,
  separators=["\n\n", "\n", ".", " ", ""],
  length_function=len
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=92


## 4. OpenSearch Vector Indexer 생성

### Connect to OpenSearch

In [43]:
index_name = "genai-demo-index-v1"
# opensearch_endpoint = "https://localhost:9200"


### 오픈 서치 인덱스 유무 확인 및 삭제

In [48]:
from utils.opensearch import opensearch_utils

from opensearchpy import OpenSearch

host = 'localhost'
port = 9200
opensearch_endpoint = f"https://{host}:{port}"
http_auth = ('admin', 'admin') # For testing only. Don't store credentials in code.
ca_certs_path = 'root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Optional client certificates if you don't want to use HTTP basic authentication.
client_cert_path = 'admin.pem'
client_key_path = 'admin-key.pem'

# Create the client with SSL/TLS enabled, but hostname verification disabled.
os_client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = http_auth,
    client_cert = client_cert_path,
    client_key = client_key_path,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

def check_if_index_exists(os_client, index_name):
      '''
      인덱스가 존재하는지 확인
      '''
      exists = os_client.indices.exists(index_name)
      print(f"index_name={index_name}, exists={exists}")

      return exists

def delete_index(os_client, index_name):
      response = os_client.indices.delete(
          index=index_name
      )

      print('\nDeleting index:')
      print(response)

index_exist = check_if_index_exists(os_client, index_name)

if index_exist:
      delete_index(os_client, index_name)
else:
      print("Index does not exist")

index_name=genai-demo-index-v1, exists=False
Index does not exist


In [49]:
from langchain.vectorstores import OpenSearchVectorSearch

docsearch = OpenSearchVectorSearch.from_documents(
    index_name = index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_endpoint,
    http_auth=http_auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

In [65]:
vector_db = OpenSearchVectorSearch(
  index_name=index_name,
  opensearch_url=opensearch_endpoint,
  embedding_function=llm_emb,
  http_auth=http_auth,
  is_aoss=False,
  space_type="l2",
  use_ssl = True,
  verify_certs = True,
  ssl_assert_hostname = False,
  ssl_show_warn = False,
  ca_certs = ca_certs_path
)

### 5. 유사서치 검색

In [51]:
from langchain.chains.question_answering import load_qa_chain
import copy

In [66]:
def get_semantic_similar_docs(**kwargs):
  search_types = ["approximate_search", "script_scoring", "painless_scripting"]
  space_types = ["l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit"]

  assert "vector_db" in kwargs, "Check your vector_db"
  assert "query" in kwargs, "Check your query"
  assert kwargs.get("search_type", "approximate_search") in search_types, f"Check your {space_types}"
  assert kwargs.get("space_type", "l2") in space_types, f"Check your space_type: {space_types}"

  results = kwargs["vector_db"].similarity_search_with_score(
    query=kwargs["query"],
    k=kwargs.get("k", 5),
    search_type=kwargs.get("search_type","approximate_search"),
    space_type=kwargs.get("space_type", "l2"),
    boolean_filter = kwargs.get("boolean_filter", {})
  )

  if kwargs.get("hybrid",False):
    max_score = results[0][1]
    new_results = []
    for doc in results:
      nomalized_score = float(doc[1]/max_score)
      new_results.append((doc[0], nomalized_score))
    results = copy.deepcopy(new_results)
  return results

### Filter 없이 실행

In [67]:
query = '간편조회서비스는 회원가입해야 하나요?'

pre_similar_doc = get_semantic_similar_docs(
  vector_db=vector_db,
  query=query,
  k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)


Score: 0.0060073393
Document Number: 20
ask: 간편조회서비스는 회원가입해야만 이용할 수 있나요?
Information: 간편조회서비스에는 로그인을 위해 회원가입이 필요한 서비스와 회원가입 없이 누구나 이용 가능한 서비스가 있습니다.
Metadata:
Type: 간편서비스
Source: 신한은행
--------------------------------------------------

Score: 0.004864754
Document Number: 49
ask: 홈페이지 회원가입은 어떻게 해야 하나요?
Information: 홈페이지 상단에 있는 [고객센터]를 클릭하여, 회원서비스 → 회원가입메뉴에서 회원가입 메뉴를 이용하시기 바랍니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
Metadata:
Type: 홈페이지
Source: 신한은행
--------------------------------------------------

Score: 0.004693966
Document Number: 19
ask: 아이핀으로 홈페이지회원 가입한 고객은 간편조회서비스 이용할 수 없나요?
Information: 아이핀으로 홈페이지 회원가입하신 고객은 홈페이지에서 개인회원으로 전환 후 간편조회서비스 이용가능 합니다. 
Metadata:
Type: 간편서비스
Source: 신한은행
--------------------------------------------------


### metadata의 type, source에 필터를 걸어서 검색

In [68]:
boolean_filter = opensearch_utils.get_filter(
  filter=[
    {"term": {"metadata.source": "신한은행"}},
    {"term": {"metadata.type": "인터넷뱅킹"}}
  ]
)
print(boolean_filter)

{'bool': {'filter': [{'term': {'metadata.source': '신한은행'}}, {'term': {'metadata.type': '인터넷뱅킹'}}]}}


In [71]:
filter01 = "인터넷뱅킹"
#filter01 = "인증서"
filter02 = "신한은행"
#filter02 = "아마존은행"

boolean_filter = opensearch_utils.get_filter(
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ]
)

pre_similar_doc = get_semantic_similar_docs(
  vector_db=vector_db,
  query=query,
  boolean_filter=boolean_filter,
  k=3
)
opensearch_utils.opensearch_pretty_print_documents(pre_similar_doc)

{'bool': {'filter': [{'term': {'metadata.type': '인터넷뱅킹'}}, {'term': {'metadata.source': '신한은행'}}]}}
